In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error,roc_auc_score,roc_curve,classification_report,confusion_matrix,plot_confusion_matrix
from statsmodels.stats.outliers_influence import variance_inflation_factor
import math
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Basic packages
from scipy import stats; from scipy.stats import zscore, norm, randint
import matplotlib.style as style; style.use('fivethirtyeight')
#import plotly.express as px
%matplotlib inline

# Impute and Encode
from sklearn.preprocessing import LabelEncoder
#from impute.imputation.cs import mice

# Modelling - LR, KNN, NB, Metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score,roc_curve

# Oversampling
from imblearn.over_sampling import SMOTE

# Suppress warnings
import warnings; warnings.filterwarnings('ignore')

# Visualize Tree
from sklearn.tree import export_graphviz
from IPython.display import Image
from os import system

# Display settings
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000

In [90]:


# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
 
 
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)
# link = 'https://drive.google.com/file/d/1D4z13rFbwe6ch0WwEiCO2dd5a18avot6/view?usp=sharing'
# link2 = 'https://drive.google.com/file/d/1qLbITeJToHCKtzG2HnvSZUn5v3LeW7lg/view?usp=sharing'
 
# import pandas as pd
 
# # to get the id part of the file
# id = link.split("/")[-2]
# id2 = link2.split("/")[-2]
 
# downloaded = drive.CreateFile({'id':id})
# downloaded.GetContentFile('Train_Set_TVS.csv') 
df_train = pd.read_csv("Train_Set_TVS.csv")
df_train.head()

,ID,Compensation,Vehicle_Status,Loan_amt,Annuity_amt,density_Ind,Job_Type,Default_hist,Prop_List,Home_Appliances,Region_Type,Credit_Inq,Education_det,Marital_Stat,Gen,Credit_Lnks,Experience,Income_Alt,Day_Process,Ph.No,Type_Loan,Dependents,Company_Rating,Region,Age,hour_Process,Addr_permanent,Permanent_Addr_tag,c1_rated,c2_rated,Contact_age,Product_bought,Default
0,540071,801850.0,Y,658040.0,13816.52,0.0304,IT,ndf,Morethan_One,NaN,Metro,1,Grad,Married,F,1_to_3,0.9,Not_Specified,SAT,Yes,CLoan,3.0,2.0,East,46.0,10.0,Yes,No,0.077,0.077,18.0,2.0,1.0
1,511643,884222.0,N,198075.0,11173.06,0.0194,IT,ndf,Morethan_One,1.0,Metro,2,Grad,Married,M,1_to_3,3.0,Not_Specified,wed,Yes,CLoan,3.0,1.0,North,46.0,12.0,Yes,Yes,0.184,0.184,306.0,4.0,0.0
2,506688,284120.0,N,101056.0,8390.40,0.0118,Govt.,ndf,One,4.0,Metro,3,Other,Married,M,1_to_3,NaN,Not_Specified,mon,Yes,CLoan,2.0,3.0,North,57.0,5.0,Yes,Yes,0.538,0.538,281.0,4.0,0.0
3,499360,341874.0,N,179008.0,7184.13,0.0373,Non-IT,df,None,2.0,Metro,1,Grad,Married,M,1_to_3,4.7,Not_Specified,wed,Yes,RLoan,4.0,2.0,South,39.0,12.0,Yes,Yes,NaN,0.561,20.0,4.0,0.0
4,438350,1259895.0,Y,573715.0,13212.77,0.0138,Non-IT,ndf,Morethan_One,1.0,Rural,2,Grad,Married,F,4_to_6,6.9,Y,Sun,Yes,CLoan,3.0,1.0,North,32.0,16.0,Yes,Yes,NaN,0.200,2339.0,6.0,0.0


In [3]:
df_train.shape

(132440, 33)

In [4]:
df_train.describe()

,ID,Compensation,Loan_amt,Annuity_amt,density_Ind,Home_Appliances,Experience,Dependents,Company_Rating,Age,hour_Process,c1_rated,c2_rated,Contact_age,Product_bought,Default
count,132440.000000,1.261980e+05,1.324400e+05,127829.000000,127024.000000,121476.000000,110595.000000,132440.000000,120709.000000,132440.00000,132440.000000,114927.000000,119119.000000,124287.000000,132440.000000,132440.000000
mean,488938.279319,7.450519e+05,4.148011e+05,10845.286442,0.020446,3.220891,6.580616,2.155467,2.030553,43.93326,12.060865,0.505929,0.506181,979.451383,3.387738,0.264301
std,58568.207772,4.702762e+05,2.746788e+05,2061.529355,0.015077,1.241751,6.357054,0.909046,0.503750,11.96270,3.566714,0.268665,0.268716,829.632194,2.173246,0.440962
min,387652.000000,1.527280e+05,3.427500e+04,5852.100000,0.000100,1.000000,0.000000,1.000000,1.000000,21.00000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000
25%,438095.500000,5.177060e+05,2.045300e+05,9348.370000,0.010400,2.000000,2.100000,2.000000,2.000000,34.00000,10.000000,0.268000,0.268000,288.000000,2.000000,0.000000
50%,488952.500000,6.659960e+05,3.495425e+05,10773.820000,0.020400,4.000000,4.600000,2.000000,2.000000,43.00000,12.000000,0.500000,0.500000,770.000000,3.000000,0.000000
75%,539702.000000,8.794890e+05,5.582050e+05,12165.390000,0.030400,4.000000,9.000000,3.000000,2.000000,54.00000,15.000000,0.769000,0.769000,1589.000000,5.000000,1.000000
max,590407.000000,7.037702e+07,2.872142e+06,41301.000000,2.017800,19.000000,48.100000,9.000000,3.000000,69.00000,25.000000,0.947000,0.947000,4309.000000,25.000000,1.000000


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132440 entries, 0 to 132439
Data columns (total 33 columns):
ID                    132440 non-null int64
Compensation          126198 non-null float64
Vehicle_Status        123336 non-null object
Loan_amt              132440 non-null float64
Annuity_amt           127829 non-null float64
density_Ind           127024 non-null float64
Job_Type              122192 non-null object
Default_hist          132440 non-null object
Prop_List             130475 non-null object
Home_Appliances       121476 non-null float64
Region_Type           132440 non-null object
Credit_Inq            129750 non-null object
Education_det         132440 non-null object
Marital_Stat          132440 non-null object
Gen                   132440 non-null object
Credit_Lnks           132440 non-null object
Experience            110595 non-null float64
Income_Alt            124758 non-null object
Day_Process           132440 non-null object
Ph.No                 132440 

In [6]:
df_train.isnull().sum()

ID                        0
Compensation           6242
Vehicle_Status         9104
Loan_amt                  0
Annuity_amt            4611
density_Ind            5416
Job_Type              10248
Default_hist              0
Prop_List              1965
Home_Appliances       10964
Region_Type               0
Credit_Inq             2690
Education_det             0
Marital_Stat              0
Gen                       0
Credit_Lnks               0
Experience            21845
Income_Alt             7682
Day_Process               0
Ph.No                     0
Type_Loan                 0
Dependents                0
Company_Rating        11731
Region                    0
Age                       0
hour_Process              0
Addr_permanent            0
Permanent_Addr_tag        0
c1_rated              17513
c2_rated              13321
Contact_age            8153
Product_bought            0
Default                   0
dtype: int64

In [7]:
(df_train.isnull().sum()/len(df_train))*100

ID                     0.000000
Compensation           4.713078
Vehicle_Status         6.874056
Loan_amt               0.000000
Annuity_amt            3.481577
density_Ind            4.089399
Job_Type               7.737844
Default_hist           0.000000
Prop_List              1.483691
Home_Appliances        8.278466
Region_Type            0.000000
Credit_Inq             2.031108
Education_det          0.000000
Marital_Stat           0.000000
Gen                    0.000000
Credit_Lnks            0.000000
Experience            16.494262
Income_Alt             5.800362
Day_Process            0.000000
Ph.No                  0.000000
Type_Loan              0.000000
Dependents             0.000000
Company_Rating         8.857596
Region                 0.000000
Age                    0.000000
hour_Process           0.000000
Addr_permanent         0.000000
Permanent_Addr_tag     0.000000
c1_rated              13.223346
c2_rated              10.058140
Contact_age            6.155995
Product_

### Droping variables 'Experience','c1_rated','c2_rated' having null greater than 10% of the data.

In [91]:
df_train = df_train.drop(['Experience','c1_rated','c2_rated'],axis=1)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132440 entries, 0 to 132439
Data columns (total 30 columns):
ID                    132440 non-null int64
Compensation          126198 non-null float64
Vehicle_Status        123336 non-null object
Loan_amt              132440 non-null float64
Annuity_amt           127829 non-null float64
density_Ind           127024 non-null float64
Job_Type              122192 non-null object
Default_hist          132440 non-null object
Prop_List             130475 non-null object
Home_Appliances       121476 non-null float64
Region_Type           132440 non-null object
Credit_Inq            129750 non-null object
Education_det         132440 non-null object
Marital_Stat          132440 non-null object
Gen                   132440 non-null object
Credit_Lnks           132440 non-null object
Income_Alt            124758 non-null object
Day_Process           132440 non-null object
Ph.No                 132440 non-null object
Type_Loan             132440 n

In [92]:
for column in df_train.columns:
    if df_train[column].dtype == 'object':
        print("Unique values in ",column.upper(),'is : ',df_train[column].nunique())
        print(df_train[column].value_counts().sort_values())
        print('\n')

Unique values in  VEHICLE_STATUS is :  2
Y    41966
N    81370
Name: Vehicle_Status, dtype: int64


Unique values in  JOB_TYPE is :  3
Govt.     25722
Non-IT    29488
IT        66982
Name: Job_Type, dtype: int64


Unique values in  DEFAULT_HIST is :  2
df      13270
ndf    119170
Name: Default_hist, dtype: int64


Unique values in  PROP_LIST is :  3
None            13020
Morethan_One    25896
One             91559
Name: Prop_List, dtype: int64


Unique values in  REGION_TYPE is :  3
other      2614
Rural     22278
Metro    107548
Name: Region_Type, dtype: int64


Unique values in  CREDIT_INQ is :  4
>4     9138
3     23178
2     30362
1     67072
Name: Credit_Inq, dtype: int64


Unique values in  EDUCATION_DET is :  3
Other      6125
P_Grad    32457
Grad      93858
Name: Education_det, dtype: int64


Unique values in  MARITAL_STAT is :  4
Widow       6776
Divorce     8391
Single     19664
Married    97609
Name: Marital_Stat, dtype: int64


Unique values in  GEN is :  2
F    45377
M    

In [93]:
df_train.columns = df_train.columns.str.replace('.', '_')
df_train.columns

Index(['ID', 'Compensation', 'Vehicle_Status', 'Loan_amt', 'Annuity_amt',
       'density_Ind', 'Job_Type', 'Default_hist', 'Prop_List',
       'Home_Appliances', 'Region_Type', 'Credit_Inq', 'Education_det',
       'Marital_Stat', 'Gen', 'Credit_Lnks', 'Income_Alt', 'Day_Process',
       'Ph_No', 'Type_Loan', 'Dependents', 'Company_Rating', 'Region', 'Age',
       'hour_Process', 'Addr_permanent', 'Permanent_Addr_tag', 'Contact_age',
       'Product_bought', 'Default'],
      dtype='object')

In [ ]:
splot_cols=df_train.columns
for i in splot_cols:
    if df_train[i].dtype != 'object' and df_train[i].isnull().values.any() != True:
        sns.boxplot(df_train[i])
        plt.show()

In [ ]:
df_train.hist(figsize=(20,30));

In [ ]:
sns.pairplot(df_train,diag_kind='kde',palette="tab10")

In [ ]:
df_train.corr()

In [ ]:
plt.figure(figsize=(20,10)) sns.heatmap(df_train.corr(), annot=True,mask=np.triu(df_train.corr(),+1))

### Treating Null values

In [94]:
for column in df_train.columns:
    if df_train[column].dtype != 'object':
        median = df_train[column].median()
        df_train[column] = df_train[column].fillna(median)    
        
df_train.isnull().sum() 

ID                        0
Compensation              0
Vehicle_Status         9104
Loan_amt                  0
Annuity_amt               0
density_Ind               0
Job_Type              10248
Default_hist              0
Prop_List              1965
Home_Appliances           0
Region_Type               0
Credit_Inq             2690
Education_det             0
Marital_Stat              0
Gen                       0
Credit_Lnks               0
Income_Alt             7682
Day_Process               0
Ph_No                     0
Type_Loan                 0
Dependents                0
Company_Rating            0
Region                    0
Age                       0
hour_Process              0
Addr_permanent            0
Permanent_Addr_tag        0
Contact_age               0
Product_bought            0
Default                   0
dtype: int64

In [95]:
for column in df_train:
    if df_train[column].dtype == 'object':
        if df_train[column].isnull().any():
#             if(column in cateogry_columns):
                df_train[column]=df_train[column].fillna(df_train[column].mode()[0])
df_train.isnull().sum() 

ID                    0
Compensation          0
Vehicle_Status        0
Loan_amt              0
Annuity_amt           0
density_Ind           0
Job_Type              0
Default_hist          0
Prop_List             0
Home_Appliances       0
Region_Type           0
Credit_Inq            0
Education_det         0
Marital_Stat          0
Gen                   0
Credit_Lnks           0
Income_Alt            0
Day_Process           0
Ph_No                 0
Type_Loan             0
Dependents            0
Company_Rating        0
Region                0
Age                   0
hour_Process          0
Addr_permanent        0
Permanent_Addr_tag    0
Contact_age           0
Product_bought        0
Default               0
dtype: int64

In [96]:
df_train.head()

,ID,Compensation,Vehicle_Status,Loan_amt,Annuity_amt,density_Ind,Job_Type,Default_hist,Prop_List,Home_Appliances,Region_Type,Credit_Inq,Education_det,Marital_Stat,Gen,Credit_Lnks,Income_Alt,Day_Process,Ph_No,Type_Loan,Dependents,Company_Rating,Region,Age,hour_Process,Addr_permanent,Permanent_Addr_tag,Contact_age,Product_bought,Default
0,540071,801850.0,Y,658040.0,13816.52,0.0304,IT,ndf,Morethan_One,4.0,Metro,1,Grad,Married,F,1_to_3,Not_Specified,SAT,Yes,CLoan,3.0,2.0,East,46.0,10.0,Yes,No,18.0,2.0,1.0
1,511643,884222.0,N,198075.0,11173.06,0.0194,IT,ndf,Morethan_One,1.0,Metro,2,Grad,Married,M,1_to_3,Not_Specified,wed,Yes,CLoan,3.0,1.0,North,46.0,12.0,Yes,Yes,306.0,4.0,0.0
2,506688,284120.0,N,101056.0,8390.40,0.0118,Govt.,ndf,One,4.0,Metro,3,Other,Married,M,1_to_3,Not_Specified,mon,Yes,CLoan,2.0,3.0,North,57.0,5.0,Yes,Yes,281.0,4.0,0.0
3,499360,341874.0,N,179008.0,7184.13,0.0373,Non-IT,df,None,2.0,Metro,1,Grad,Married,M,1_to_3,Not_Specified,wed,Yes,RLoan,4.0,2.0,South,39.0,12.0,Yes,Yes,20.0,4.0,0.0
4,438350,1259895.0,Y,573715.0,13212.77,0.0138,Non-IT,ndf,Morethan_One,1.0,Rural,2,Grad,Married,F,4_to_6,Y,Sun,Yes,CLoan,3.0,1.0,North,32.0,16.0,Yes,Yes,2339.0,6.0,0.0


In [97]:
df_train['Default'] = df_train['Default'].astype('int64')

In [98]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132440 entries, 0 to 132439
Data columns (total 30 columns):
ID                    132440 non-null int64
Compensation          132440 non-null float64
Vehicle_Status        132440 non-null object
Loan_amt              132440 non-null float64
Annuity_amt           132440 non-null float64
density_Ind           132440 non-null float64
Job_Type              132440 non-null object
Default_hist          132440 non-null object
Prop_List             132440 non-null object
Home_Appliances       132440 non-null float64
Region_Type           132440 non-null object
Credit_Inq            132440 non-null object
Education_det         132440 non-null object
Marital_Stat          132440 non-null object
Gen                   132440 non-null object
Credit_Lnks           132440 non-null object
Income_Alt            132440 non-null object
Day_Process           132440 non-null object
Ph_No                 132440 non-null object
Type_Loan             132440 n

In [15]:
df_train.head()

,ID,Compensation,Vehicle_Status,Loan_amt,Annuity_amt,density_Ind,Job_Type,Default_hist,Prop_List,Home_Appliances,Region_Type,Credit_Inq,Education_det,Marital_Stat,Gen,Credit_Lnks,Income_Alt,Day_Process,Ph_No,Type_Loan,Dependents,Company_Rating,Region,Age,hour_Process,Addr_permanent,Permanent_Addr_tag,Contact_age,Product_bought,Default
0,540071,801850.0,Y,658040.0,13816.52,0.0304,IT,ndf,Morethan_One,4.0,Metro,1,Grad,Married,F,1_to_3,Not_Specified,SAT,Yes,CLoan,3.0,2.0,East,46.0,10.0,Yes,No,18.0,2.0,1
1,511643,884222.0,N,198075.0,11173.06,0.0194,IT,ndf,Morethan_One,1.0,Metro,2,Grad,Married,M,1_to_3,Not_Specified,wed,Yes,CLoan,3.0,1.0,North,46.0,12.0,Yes,Yes,306.0,4.0,0
2,506688,284120.0,N,101056.0,8390.40,0.0118,Govt.,ndf,One,4.0,Metro,3,Other,Married,M,1_to_3,Not_Specified,mon,Yes,CLoan,2.0,3.0,North,57.0,5.0,Yes,Yes,281.0,4.0,0
3,499360,341874.0,N,179008.0,7184.13,0.0373,Non-IT,df,None,2.0,Metro,1,Grad,Married,M,1_to_3,Not_Specified,wed,Yes,RLoan,4.0,2.0,South,39.0,12.0,Yes,Yes,20.0,4.0,0
4,438350,1259895.0,Y,573715.0,13212.77,0.0138,Non-IT,ndf,Morethan_One,1.0,Rural,2,Grad,Married,F,4_to_6,Y,Sun,Yes,CLoan,3.0,1.0,North,32.0,16.0,Yes,Yes,2339.0,6.0,0


In [99]:
df_cat_train = df_train.select_dtypes(include = ['object'])
df_num_train = df_train.select_dtypes(include = ['float64','int64'])

In [100]:
df_cat_train = pd.get_dummies(df_cat_train, drop_first = True)
df_cat_train.head()

,Vehicle_Status_Y,Job_Type_IT,Job_Type_Non-IT,Default_hist_ndf,Prop_List_None,Prop_List_One,Region_Type_Rural,Region_Type_other,Credit_Inq_2,Credit_Inq_3,Credit_Inq_>4,Education_det_Other,Education_det_P_Grad,Marital_Stat_Married,Marital_Stat_Single,Marital_Stat_Widow,Gen_M,Credit_Lnks_4_to_6,Credit_Lnks_>6,Income_Alt_Y,Day_Process_SAT,Day_Process_Sun,Day_Process_Tue,Day_Process_mon,Day_Process_thur,Day_Process_wed,Ph_No_Yes,Type_Loan_RLoan,Region_East,Region_North,Region_South,Addr_permanent_Yes,Permanent_Addr_tag_Yes
0,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,1
2,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1
3,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1
4,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,1,1


In [103]:
# df_train_copy1 = pd.concat([df_cat_train,df_num_train],axis=1)
df_train_copy2 = pd.concat([df_num_train,df_cat_train],axis=1)
df_train_copy2.head()

,ID,Compensation,Loan_amt,Annuity_amt,density_Ind,Home_Appliances,Dependents,Company_Rating,Age,hour_Process,Contact_age,Product_bought,Default,Vehicle_Status_Y,Job_Type_IT,Job_Type_Non-IT,Default_hist_ndf,Prop_List_None,Prop_List_One,Region_Type_Rural,Region_Type_other,Credit_Inq_2,Credit_Inq_3,Credit_Inq_>4,Education_det_Other,Education_det_P_Grad,Marital_Stat_Married,Marital_Stat_Single,Marital_Stat_Widow,Gen_M,Credit_Lnks_4_to_6,Credit_Lnks_>6,Income_Alt_Y,Day_Process_SAT,Day_Process_Sun,Day_Process_Tue,Day_Process_mon,Day_Process_thur,Day_Process_wed,Ph_No_Yes,Type_Loan_RLoan,Region_East,Region_North,Region_South,Addr_permanent_Yes,Permanent_Addr_tag_Yes
0,540071,801850.0,658040.0,13816.52,0.0304,4.0,3.0,2.0,46.0,10.0,18.0,2.0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0
1,511643,884222.0,198075.0,11173.06,0.0194,1.0,3.0,1.0,46.0,12.0,306.0,4.0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,1
2,506688,284120.0,101056.0,8390.40,0.0118,4.0,2.0,3.0,57.0,5.0,281.0,4.0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1
3,499360,341874.0,179008.0,7184.13,0.0373,2.0,4.0,2.0,39.0,12.0,20.0,4.0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1
4,438350,1259895.0,573715.0,13212.77,0.0138,1.0,3.0,1.0,32.0,16.0,2339.0,6.0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,1,1


In [104]:
# df_train_copy1 = df_train_copy1.drop(columns=['ID'], axis=1)
df_train_copy2 = df_train_copy2.drop(columns=['ID'], axis=1)
df_train_copy2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132440 entries, 0 to 132439
Data columns (total 45 columns):
Compensation              132440 non-null float64
Loan_amt                  132440 non-null float64
Annuity_amt               132440 non-null float64
density_Ind               132440 non-null float64
Home_Appliances           132440 non-null float64
Dependents                132440 non-null float64
Company_Rating            132440 non-null float64
Age                       132440 non-null float64
hour_Process              132440 non-null float64
Contact_age               132440 non-null float64
Product_bought            132440 non-null float64
Default                   132440 non-null int64
Vehicle_Status_Y          132440 non-null uint8
Job_Type_IT               132440 non-null uint8
Job_Type_Non-IT           132440 non-null uint8
Default_hist_ndf          132440 non-null uint8
Prop_List_None            132440 non-null uint8
Prop_List_One             132440 non-null uint8
Reg

In [20]:
# x = df_train_copy1.drop('Default', axis=1)

# y = df_train_copy1['Default']

In [105]:
x = df_train_copy2.drop('Default', axis=1)

y = df_train_copy2['Default']

In [106]:
# from scipy.stats import zscore
# x2 = x2.apply(zscore)
# # train_scaled = pd.concat([df_cat_train,df_num_train],axis=1)
# x2.head()
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(x.to_numpy())
df_scaled = pd.DataFrame(df_scaled, columns=[x2.columns])
df_scaled.head()

,Vehicle_Status_Y,Job_Type_IT,Job_Type_Non-IT,Default_hist_ndf,Prop_List_None,Prop_List_One,Region_Type_Rural,Region_Type_other,Credit_Inq_2,Credit_Inq_3,Credit_Inq_>4,Education_det_Other,Education_det_P_Grad,Marital_Stat_Married,Marital_Stat_Single,Marital_Stat_Widow,Gen_M,Credit_Lnks_4_to_6,Credit_Lnks_>6,Income_Alt_Y,Day_Process_SAT,Day_Process_Sun,Day_Process_Tue,Day_Process_mon,Day_Process_thur,Day_Process_wed,Ph_No_Yes,Type_Loan_RLoan,Region_East,Region_North,Region_South,Addr_permanent_Yes,Permanent_Addr_tag_Yes,Compensation,Loan_amt,Annuity_amt,density_Ind,Home_Appliances,Dependents,Company_Rating,Age,hour_Process,Contact_age,Product_bought
0,0.009244,0.219801,0.224673,0.015017,0.166667,0.250,0.5,0.520833,0.40,0.001861,0.08,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.010417,0.057719,0.150102,0.009565,0.000000,0.250,0.0,0.520833,0.48,0.068853,0.16,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
2,0.001871,0.023532,0.071604,0.005799,0.166667,0.125,1.0,0.750000,0.20,0.063038,0.16,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
3,0.002693,0.051001,0.037576,0.018437,0.055556,0.375,0.5,0.375000,0.48,0.002326,0.16,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0
4,0.015766,0.190086,0.207642,0.006790,0.000000,0.250,0.0,0.229167,0.64,0.541754,0.24,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0


In [107]:
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2 , random_state=42,stratify=y)
# x_train.shape

In [108]:
x_train, x_test, y_train, y_test = train_test_split(df_scaled, y, test_size=0.2 , random_state=42,stratify=y)
x_train.shape

(105952, 44)

In [109]:
y_train.shape

(105952,)

In [110]:
x_test.shape

(26488, 44)

In [111]:
y_test.shape

(26488,)

### Logistic Regression

In [25]:
# grid={'penalty':['l1','l2','none'],
#       'solver':['lbfgs', 'liblinear'],
#       'tol':[0.0001,0.000001]}

In [112]:
grid2={'penalty':['l1','l2','none'],
      'solver':[ 'lbfgs','liblinear'],
      'tol':[0.0001,0.000001]}

In [113]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=100000,n_jobs=2)

In [27]:
# grid_search = GridSearchCV(estimator = model, param_grid = grid, cv = 3,n_jobs=-1,scoring='f1')
# grid_search.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=LogisticRegression(max_iter=100000, n_jobs=2),
             n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'none'],
                         'solver': ['lbfgs', 'liblinear'],
                         'tol': [0.0001, 1e-06]},
             scoring='f1')

In [114]:
grid_search = GridSearchCV(estimator = model, param_grid = grid2, cv = 3,n_jobs=-1,scoring='f1')
grid_search.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=LogisticRegression(max_iter=100000, n_jobs=2),
             n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'none'],
                         'solver': ['lbfgs', 'liblinear'],
                         'tol': [0.0001, 1e-06]},
             scoring='f1')

In [115]:
grid_search.best_params_

{'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.0001}

In [116]:
best_model = grid_search.best_estimator_
best_model

LogisticRegression(max_iter=100000, n_jobs=2, penalty='l1', solver='liblinear')

In [117]:
ytrain_predict = best_model.predict(x_train)
ytrain_predict

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [118]:
ytest_predict = best_model.predict(x_test)
ytest_predict

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [87]:
Logistic_regression = grid_search.score(x_test, y_test)
Logistic_regression

0.41004972860532385

In [119]:
Logistic_regression = grid_search.score(x_test, y_test)
Logistic_regression

0.0

In [73]:
print(classification_report(y_train, ytrain_predict),'\n')

              precision    recall  f1-score   support

           0       0.74      0.10      0.18     77949
           1       0.26      0.90      0.41     28003

    accuracy                           0.31    105952
   macro avg       0.50      0.50      0.29    105952
weighted avg       0.62      0.31      0.24    105952
 



In [120]:
print(classification_report(y_train, ytrain_predict),'\n')

              precision    recall  f1-score   support

           0       0.74      1.00      0.85     77949
           1       0.00      0.00      0.00     28003

    accuracy                           0.74    105952
   macro avg       0.37      0.50      0.42    105952
weighted avg       0.54      0.74      0.62    105952
 



In [74]:
print(classification_report(y_test, ytest_predict),'\n')

              precision    recall  f1-score   support

           0       0.75      0.10      0.18     19487
           1       0.27      0.90      0.41      7001

    accuracy                           0.31     26488
   macro avg       0.51      0.50      0.30     26488
weighted avg       0.62      0.31      0.24     26488
 



In [121]:
print(classification_report(y_test, ytest_predict),'\n')

              precision    recall  f1-score   support

           0       0.74      1.00      0.85     19487
           1       0.00      0.00      0.00      7001

    accuracy                           0.74     26488
   macro avg       0.37      0.50      0.42     26488
weighted avg       0.54      0.74      0.62     26488
 



In [83]:
lr_metrics=classification_report(y_test, ytest_predict,output_dict=True)
lr_metrics1=classification_report(y_train, ytrain_predict,output_dict=True)

dftra=pd.DataFrame(lr_metrics).transpose()
dftra1=pd.DataFrame(lr_metrics1).transpose()
lr_test_f1=round(dftra.loc["1"][0],2)
lr_train_f1=round(dftra1.loc["1"][0],2)
print ('lr_test_f1 ',lr_test_f1)
print ('lr_train_f1 ',lr_train_f1)

lr_test_f1  0.41


In [122]:
lr_metrics=classification_report(y_test, ytest_predict,output_dict=True)
lr_metrics1=classification_report(y_train, ytrain_predict,output_dict=True)

dftra=pd.DataFrame(lr_metrics).transpose()
dftra1=pd.DataFrame(lr_metrics1).transpose()
lr_test_f1=round(dftra.loc["1"][0],2)
lr_train_f1=round(dftra1.loc["1"][0],2)
print ('lr_test_f1 ',lr_test_f1)
print ('lr_train_f1 ',lr_train_f1)

lr_test_f1  0.0
lr_train_f1  0.0


In [84]:
resultsDf = pd.DataFrame({'f1 test':lr_test_f1,'f1 train': lr_train_f1},index=['Logistic Regression'])
resultsDf

,f1
Logistic Regression,0.41


### Random Forest

In [123]:
param_grid_RF = {
    'max_depth': [150,175,200],
    'max_features': [40,50,60],
    'min_samples_leaf': [100,150,200],
    'min_samples_split': [100,150,200],
    'n_estimators': [50,70] 
}

In [ ]:
rfcl = RandomForestClassifier(random_state=42)

rfcl = GridSearchCV(estimator = rfcl, param_grid = param_grid_RF, cv = 3) 
rfcl = rfcl.fit(x_train, y_train)

In [ ]:
rfcl.best_params_

In [ ]:
rfcl = rfcl.best_estimator_

In [ ]:
ytrain_predict = rfcl.predict(x_train)
ytrain_predict

In [ ]:
ytest_predict = rfcl.predict(x_test)
ytest_predict

In [ ]:
print(classification_report(y_train, ytrain_predict),'\n')

In [ ]:
print(classification_report(y_test, ytest_predict),'\n')

In [ ]:
rf_metrics=classification_report(y_test, ytest_predict,output_dict=True)
rf_metrics1=classification_report(y_train, ytrain_predict,output_dict=True)
dftrf=pd.DataFrame(rf_metrics).transpose()
dftrf1=pd.DataFrame(rf_metrics1).transpose()
rf_test_f1=round(dftrf.loc["1"][0],2)
rf_train_f1=round(dftrf1.loc["1"][0],2)
print ('rf_test_f1 ',rf_test_f1)
print ('rf_train_f1 ',rf_train_f1)

In [86]:
resultsDf1 = pd.DataFrame({'f1 test':rf_test_f1,'f1 train':rf_train_f1},index=['Random Forest'])
resultsDf = pd.concat([resultsDf,resultsDf1],axis=0)
resultsDf

,f1
Logistic Regression,0.41
Logistic Regression,0.41


### Decision Tree classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
param_grid = {'max_features': ['auto', 'sqrt', 'log2'],
              'ccp_alpha': [0.1, .01, .001],
              'max_depth' : [5, 6, 7, 8, 9],
              'criterion' :['gini', 'entropy']
             }
tree_clas = DecisionTreeClassifier(random_state=1024)
grid_search = GridSearchCV(estimator=tree_clas, param_grid=param_grid, cv=5, verbose=True)
grid_search.fit(x_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search = grid_search.best_estimator_

In [ ]:
ytrain_predict = grid_search.predict(x_train)
ytrain_predict

In [ ]:
ytest_predict = grid_search.predict(x_test)
ytest_predict

In [ ]:
print(classification_report(y_train, ytrain_predict),'\n')

In [ ]:
print(classification_report(y_test, ytest_predict),'\n')

In [ ]:
dt_metrics=classification_report(y_test, ytest_predict,output_dict=True)
dt_metrics1=classification_report(y_train, ytrain_predict,output_dict=True)
dt=pd.DataFrame(dt_metrics).transpose()
dt1=pd.DataFrame(dt_metrics1).transpose()
dt_test_f1=round(dt.loc["1"][0],2)
dt_train_f1=round(dt1.loc["1"][0],2)
print ('dt_test_f1 ',dt_test_f1)
print ('dt_train_f1 ',dt_train_f1)

In [ ]:
resultsDf1 = pd.DataFrame({'f1 test':dt_test_f1,'f1 train':dt_train_f1},index=['Decision Tree'])
resultsDf = pd.concat([resultsDf,resultsDf1],axis=0)
resultsDf